In [1]:
#本脚本主要实现内容为基于tripadvisor澳门地区酒店数据图像识别内容之后文本内容的数据预处理部分

In [83]:
#　导入必要的包
import csv
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster,cut_tree,cophenet,inconsistent
import scipy.cluster.hierarchy as sch
from matplotlib import pyplot as plt
from matplotlib.pyplot import xticks
import pandas as pd
from scipy.spatial import distance
from itertools import groupby #itertool还包含有其他很多函数，比如将多个list联合起来
import json
from sklearn.metrics.pairwise import euclidean_distances
import re
from orangecontrib.associate.fpgrowth import * 
from collections import Counter
from gensim.models import KeyedVectors

In [84]:
'''
函数名：读数据
输入：json数据的路径
输出：每张图片用list数据存储的label集，每个图片label的评分集，以及每个图片的url
描述：该函数实现从文件中读取json数据文件，并分别解析出label集，评分集，以及url集
'''
def loadjson_label(path):
    with open (path,'rb') as json_data:
        data = json.load(json_data)
    labels = []
    
    #打印所有的labels
    for i in data:
        labels.append(i[:-1])
    classes = []
    for i in labels:
        temp = []
        for j in i:
            c = list(j[0].values())
            temp.append(c[0])
        classes.append(temp)
    
    # 打印所有的score
    score = []
    for i in labels:
        temp = []
        for j in i:
            c = list(j[1].values())
            temp.append(c[0])
        score.append(temp)

    
    #打印所有的urls
    urls = []
    for i in data:
        urls.append(list(i[-1].values())[0])    
    return classes, score, urls

In [85]:
# 读数据face
def loadjson_face(path):
    with open (path,'rb') as json_data:
        data = json.load(json_data)
    data_with_face = []
    for i in data:
        if len(i)!=1:
            data_with_face.append(i)
    rate = len(data_with_face)/len(data)
#     print ("识别出来脸总共：",len(data_with_face))
#     print ("总数据：",len(data))
    return data,data_with_face,rate

In [86]:
# 读数据color
def loadjson_color(path):
    with open (path,'rb') as json_data:
        data = json.load(json_data)
    labels = []
    # #打印所有的了labels
    for i in data:
        labels.append(i[:-1])
    temp = []
    for i in labels:
        temp_t = []
        for j in i:
            temp_t.append(j[:4])
        temp.append(temp_t)

    urls = []
    for i in data:
        urls.append(i[-1])

    datasets = []
    for i in range(0,len(temp)):
        datasets.append(list(temp[i])+list(urls[i].values()))
        
    return datasets

In [87]:
# 读label
print('====================================官方数据集label识别结果=======================================')
official_labelset_five,official_score_five, official_urls_five = loadjson_label('dataset\official\google\label\\five.json')
print('五星级酒店图片总共:',len(official_labelset_five))
official_labelset_four,official_score_four, official_urls_four = loadjson_label('dataset\official\google\label\\four.json')
print ('四星级酒店图片总共:',len(official_labelset_four))
official_labelset_three,official_score_three, official_urls_three = loadjson_label('dataset\official\google\label\\three.json')
print ('三星级酒店图片总共:',len(official_labelset_three))
official_labelset_two,official_score_two, official_urls_two = loadjson_label('dataset\official\google\label\\two.json')
print ('二星级酒店图片总共:',len(official_labelset_two))
print('总数据量：',len(official_labelset_five)+len(official_labelset_four)+len(official_labelset_three)+len(official_labelset_two))

# 读color
print('=====================================官方数据集color识别结果========================================')
official_colorset_five = loadjson_color('dataset\official\google\color\\five.json')
print('五星级酒店可识别颜色图片总数：',len(official_colorset_five))
official_colorset_four = loadjson_color('dataset\official\google\color\\four.json')
print('四星级酒店可识别颜色图片总数：',len(official_colorset_four))
official_colorset_three= loadjson_color('dataset\official\google\color\\three.json')
print('三星级酒店可识别颜色图片总数：',len(official_colorset_three))
official_colorset_two= loadjson_color('dataset\official\google\color\\two.json')
print('二星级酒店可识别颜色图片总数：',len(official_colorset_two))
print ('总数据量：',len(official_colorset_five)+len(official_colorset_four)+len(official_colorset_three)+len(official_colorset_two))

# 读face
print ('======================================官方数据集face识别结果=======================================')
#其中faceset_five表示五星级酒店总数据，face_five表示识别出来脸的图片集，rate_five表示评分
official_faceset_five,official_face_five,official_rate_five = loadjson_face('dataset\official\google\\face\\five.json')
print('五星级酒店识别出来脸的图片：',len(official_face_five))
official_faceset_four,official_face_four,official_rate_four = loadjson_face('dataset\official\google\\face\\four.json')
print('四星级酒店识别出来脸的图片：',len(official_face_four))
official_faceset_three,official_face_three,official_rate_three = loadjson_face('dataset\official\google\\face\\three.json')
print('三星级酒店识别出来脸的图片：',len(official_face_three))
official_faceset_two,official_face_two,official_rate_two = loadjson_face('dataset\official\google\\face\\two.json')
print('二星级酒店识别出来脸的图片：',len(official_face_two))
print ('总共识别出来脸的图片数据量：',len(official_face_five)+len(official_face_four)+len(official_face_three)+len(official_face_two))

====================================官方数据集label识别结果=======================================
五星级酒店图片总共: 1221
四星级酒店图片总共: 849
三星级酒店图片总共: 222
二星级酒店图片总共: 105
总数据量： 2397
=====================================官方数据集color识别结果========================================
五星级酒店可识别颜色图片总数： 1221
四星级酒店可识别颜色图片总数： 849
三星级酒店可识别颜色图片总数： 222
二星级酒店可识别颜色图片总数： 105
总数据量： 2397
======================================官方数据集face识别结果=======================================
五星级酒店识别出来脸的图片： 26
四星级酒店识别出来脸的图片： 11
三星级酒店识别出来脸的图片： 0
二星级酒店识别出来脸的图片： 0
总共识别出来脸的图片数据量： 37


In [88]:
# 读label
print('====================================游客数据集label识别结果=======================================')
traveler_labelset_five,traveler_score_five, traveler_urls_five = loadjson_label('dataset\\travelers\google\label\\five.json')
print('五星级酒店图片总共:',len(traveler_labelset_five))
traveler_labelset_four,traveler_score_four, traveler_urls_four = loadjson_label('dataset\\travelers\google\label\\four.json')
print ('四星级酒店图片总共:',len(traveler_labelset_four))
traveler_labelset_three,traveler_score_three, traveler_urls_three = loadjson_label('dataset\\travelers\google\label\\three.json')
print ('三星级酒店图片总共:',len(traveler_labelset_three))
traveler_labelset_two,traveler_score_two, traveler_urls_two = loadjson_label('dataset\\travelers\google\label\\two.json')
print ('二星级酒店图片总共:',len(traveler_labelset_two))
print('总数据量：',len(traveler_labelset_five)+len(traveler_labelset_four)+len(traveler_labelset_three)+len(traveler_labelset_two))

# 读color
print('=====================================游客数据集color识别结果========================================')
traveler_colorset_five = loadjson_color('dataset\\travelers\google\color\\five.json')
print('五星级酒店可识别颜色图片总数：',len(traveler_colorset_five))
traveler_colorset_four = loadjson_color('dataset\\travelers\google\color\\four.json')
print('四星级酒店可识别颜色图片总数：',len(traveler_colorset_four))
traveler_colorset_three= loadjson_color('dataset\\travelers\google\color\\three.json')
print('三星级酒店可识别颜色图片总数：',len(traveler_colorset_three))
traveler_colorset_two= loadjson_color('dataset\\travelers\google\color\\two.json')
print('二星级酒店可识别颜色图片总数：',len(traveler_colorset_two))
print ('总数据量：',len(traveler_colorset_five)+len(traveler_colorset_four)+len(traveler_colorset_three)+len(traveler_colorset_two))

# 读face
print ('======================================游客数据集face识别结果=======================================')
#其中faceset_five表示五星级酒店总数据，face_five表示识别出来脸的图片集，rate_five表示评分
traveler_faceset_five,traveler_face_five,traveler_rate_five = loadjson_face('dataset\\travelers\google\\face\\five.json')
print('五星级酒店识别出来脸的图片：',len(traveler_face_five))
traveler_faceset_four,traveler_face_four,traveler_rate_four = loadjson_face('dataset\\travelers\google\\face\\four.json')
print('四星级酒店识别出来脸的图片：',len(traveler_face_four))
traveler_faceset_three,traveler_face_three,traveler_rate_three = loadjson_face('dataset\\travelers\google\\face\\three.json')
print('三星级酒店识别出来脸的图片：',len(traveler_face_three))
traveler_faceset_two,traveler_face_two,traveler_rate_two = loadjson_face('dataset\\travelers\google\\face\\two.json')
print('二星级酒店识别出来脸的图片：',len(traveler_face_two))
print ('总共识别出来脸的图片数据量：',len(traveler_face_five)+len(traveler_face_four)+len(traveler_face_three)+len(traveler_face_two))

====================================游客数据集label识别结果=======================================
五星级酒店图片总共: 32023
四星级酒店图片总共: 15249
三星级酒店图片总共: 3627
二星级酒店图片总共: 517
总数据量： 51416
=====================================游客数据集color识别结果========================================
五星级酒店可识别颜色图片总数： 32023
四星级酒店可识别颜色图片总数： 15249
三星级酒店可识别颜色图片总数： 3627
二星级酒店可识别颜色图片总数： 517
总数据量： 51416
======================================游客数据集face识别结果=======================================
五星级酒店识别出来脸的图片： 734
四星级酒店识别出来脸的图片： 309
三星级酒店识别出来脸的图片： 37
二星级酒店识别出来脸的图片： 6
总共识别出来脸的图片数据量： 1086


In [89]:
'''
函数名：数据预处理
输入：label集，url
输出：经过处理的label集，并将url并进去
描述：通过该函数将获取到的数据集进行预处理。1.去掉空格，2.去掉括号，3.将url添加上。
'''
def preprocessing(labelset,urls):
    i=0
    for labels in labelset:
        labels.append(urls[i])
        i+=1
    sets_ini = []
    for i in labelset:
        inter_sets = []
        for j in i:
#             j = j.replace(' ','')                          # 去掉空格
            j = j.split(' ')[-1] 
            inter_sets.append(re.sub('\((.*?\))', '', j))  # 去掉括号
        sets_ini.append(inter_sets)
    return sets_ini

In [53]:
#　对数据集进行预处理
print ('=======================================官方数据集预处理===============================')
official_five_ini = preprocessing(official_labelset_five,official_urls_five)
print ("五星级酒店官方图片数据集个数 = ", len (official_five_ini))
official_four_ini = preprocessing(official_labelset_four,official_urls_four)
print ("四星级酒店官方图片数据集个数 = ", len (official_four_ini))
official_three_ini = preprocessing(official_labelset_three,official_urls_three)
print ("三星级酒店官方图片数据集个数 = ", len (official_three_ini))
official_two_ini = preprocessing(official_labelset_two,official_urls_two)
print ("二星级酒店官方图片数据集个数 = ", len (official_two_ini))

print('========================================游客数据集预处理===============================')
traveler_five_ini = preprocessing(traveler_labelset_five,traveler_urls_five)
print ("五星级酒店游客图片数据集个数 = ", len (traveler_five_ini))
traveler_four_ini = preprocessing(traveler_labelset_four,traveler_urls_four)
print ("四星级酒店游客图片数据集个数 = ", len (traveler_four_ini))
traveler_three_ini = preprocessing(traveler_labelset_three,traveler_urls_three)
print ("三星级酒店游客图片数据集个数 = ", len (traveler_three_ini))
traveler_two_ini = preprocessing(traveler_labelset_two,traveler_urls_two)
print ("二星级酒店游客图片数据集个数 = ", len (traveler_two_ini))

=======================================官方数据集预处理===============================
五星级酒店官方图片数据集个数 =  1221
四星级酒店官方图片数据集个数 =  849
三星级酒店官方图片数据集个数 =  222
二星级酒店官方图片数据集个数 =  105
========================================游客数据集预处理===============================
五星级酒店游客图片数据集个数 =  32023
四星级酒店游客图片数据集个数 =  15249
三星级酒店游客图片数据集个数 =  3627
二星级酒店游客图片数据集个数 =  517


In [54]:
# 取出所有的label
'''
函数名：取出所有label并进行统计
输入：每个图片的label list
输出：每个数据集中的label集合，数据集label统计并排序之后的结果
描述：对之前的图片的label list中的label先取出来，然后进行统计每个label在整个数据集中出现的百分比
'''
def label_stat(sets_ini):
    all_sets = [] # 存放c不带url的数据集
    for i in sets_ini:
        all_sets.append(i[:-1])
    label_all = [] # 存放所有label
    for i in all_sets:
        for j in i:
            label_all.append(j)           
    #数据集长度定义
    length = len(all_sets)
    #数据集每个label计数
    c = Counter(label_all)
    stat_sets = []
    for i in c.items():
        stat_sets.append([i[0],round(i[1]/length,5)])
    # 数据集label 频率排序    
    n = len(stat_sets)
    for i in range(n-1):
        for j in range(n-i-1):
            if stat_sets[j][1]<stat_sets[j+1][1]:
                stat_sets[j],stat_sets[j+1] = stat_sets[j+1],stat_sets[j]
    return label_all, stat_sets

In [55]:
#official_label_all_five表示五星级酒店出现的标签，official_stat_five表示五星级酒店每个词以及这个词出现的频率
print('===============================官方数据集label统计=======================================')
official_label_all_five, official_stat_five = label_stat(official_five_ini)
print ("五星级酒店官方数据集label个数 = ",len(official_label_all_five))
official_label_all_four,official_stat_four = label_stat(official_four_ini)
print ("四星级酒店官方数据集label个数 = ",len(official_label_all_four))
official_label_all_three,official_stat_three = label_stat(official_three_ini)
print ("三星级酒店官方数据集label个数 = ",len(official_label_all_three))
official_label_all_two,official_stat_two = label_stat(official_two_ini)
print ("二星级酒店官方数据集label个数 = ",len(official_label_all_two))

print('==============================游客数据集label统计=======================================')
traveler_label_all_five, traveler_stat_five = label_stat(traveler_five_ini)
print ("五星级酒店游客数据集label个数 = ",len(traveler_label_all_five))
traveler_label_all_four,traveler_stat_four = label_stat(traveler_four_ini)
print ("四星级酒店游客数据集label个数 = ",len(traveler_label_all_four))
traveler_label_all_three,traveler_stat_three = label_stat(traveler_three_ini)
print ("三星级酒店游客数据集label个数 = ",len(traveler_label_all_three))
traveler_label_all_two,traveler_stat_two = label_stat(traveler_two_ini)
print ("二星级酒店游客数据集label个数 = ",len(traveler_label_all_two))

===============================官方数据集label统计=======================================
五星级酒店官方数据集label个数 =  9863
四星级酒店官方数据集label个数 =  6952
三星级酒店官方数据集label个数 =  1754
二星级酒店官方数据集label个数 =  782
==============================游客数据集label统计=======================================
五星级酒店游客数据集label个数 =  251626
四星级酒店游客数据集label个数 =  119876
三星级酒店游客数据集label个数 =  28408
二星级酒店游客数据集label个数 =  4067


In [58]:
# word_embedding 从字典中查找对应词的向量。 
'''
embed_file_name: glove file path.
word_set: test sets.
'''
def load_embedding_file(embed_file_name, word_set):
    ''' loads embedding file and returns a dictionary (word -> embedding) for the words existing in the word_set '''
    print("Loading GloVe")
    embeddings = {}
    with open(embed_file_name, 'r',encoding='utf-8') as embed_file:
        for line in embed_file:
#             print ('line=',line)
            content = line.strip().split(" ")
            word = content[0]
#             print ('word=',word)
            if word in word_set:
                embedding = np.array(content[1:], dtype=np.float32)
                embeddings[word] = embedding
    print("Loaded GloVe")
#     print ('embeddings=',embeddings)
    return embeddings

In [59]:
path = "E:\Master\Research\HotelImageMaterial\glove.840B.300d\glove.840B.300d.txt"  # word_embedding 路径
res_emb = load_embedding_file(path,official_label_all_two)
print (len(res_emb.keys()))
# print (res_emb.keys())

Loading GloVe
Loaded GloVe
81


In [60]:
print (official_label_all_two-res_emb.keys())

{'arecales'}


In [61]:
# google news
GOOGLENEWS_FILE_GLOVE = 'C:\\Users\TF\Desktop\BunCode\GoogleNews-vectors-negative300.bin'

def word_embedding_to_vector(path,word_set):
    embeddings = {}
    embeddings_process=KeyedVectors.load_word2vec_format(GOOGLENEWS_FILE_GLOVE, binary=True)
    for word in word_set:
        if word in embeddings_process.vocab:
            embeddings[word]=embeddings_process.word_vec(word)
        else:
            pass
    return embeddings

In [62]:
embedding_result = word_embedding_to_vector(GOOGLENEWS_FILE_GLOVE,official_label_all_two)
print (len(embedding_result.keys()))
# print (embedding_result)
print (len(official_label_all_two))

81
782


In [63]:
print (official_label_all_two-embedding_result.keys())

{'arecales'}
